In [ ]:
# LSHT
def lsht(dem_dir=f'{PATH}/{PROJ_TITLE}/wshed_dems'):
    """

    """
    path_list = glob.glob(f'{dem_dir}/*.tiff') # Create a list of all the paths to our watershed DEMs.
    
    def df_maker(path):
        dem = xdem.DEM(path) # Load in the DEM.
        tri = dem.terrain_ruggedness_index(window_size=3) # Make an SLRM (Simple Local Relief Model) of the DEM.  3 pix window size.

        # Next we are going to find the 10th percentile relief for the dem.
        t_data = tri.data # Get our data.
        t_data = np.asarray(t_data)
        t_valid_mask = ~np.isnan(t_data)
        t_valid = t_data[t_valid_mask] # Create a mask that is false if it is nan.  Extract valid values.
        perc_10th = np.percentile(t_valid, 10) # From valid values, get 10th percentile.
        t_bool_mask = np.where(t_valid_mask & (t_data < perc_10th), True, False) # True if data is valid and below 10th percentile. False otherwise.

        # We now repeat the process to find the 90th percentile elevation.
        e_data = dem.data
        e_data = np.asarray(e_data)
        e_valid_mask = ~np.isnan(e_data)
        e_valid = e_data[e_valid_mask] # Create a mask that is false if it is nan.  Extract valid values.
        perc_90th = np.percentile(e_valid, 90) # From valid values, get 10th percentile.
        e_bool_mask = np.where(e_valid_mask &  (e_data > perc_90th), True, False)

        # Get the name for our export csv.
        name = path.split(sep='/')
        name = name[-1][:-5]

        # Create our df.
        # First get our coordinate values.
        nrows, ncols = tri.data.shape
        rows, cols = np.indices((nrows, ncols))
        xs, ys = tri.transform * (cols, rows)

        names = [name] * len(xs.flatten())

        data = {'lon':xs.flatten(), 'lat':ys.flatten(), '10th_r':t_bool_mask.flatten(), '90th_e':e_bool_mask.flatten(), 'Wshed_name':names}
        df = pd.DataFrame(data=data)
        return df
        
    
    with ThreadPoolExecutor() as executor:
        results_iter = executor.map(df_maker, path_list)
        results = list(results_iter)

    return results

x = lsht()